<a href="https://colab.research.google.com/github/Samuel-David-Angus/unsupervised-clustering-discrete-math-questions/blob/main/clustering_word2vec_(processed_questions).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y numpy gensim scipy
!pip install --no-cache-dir numpy gensim scipy

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

import numpy as np

from google.colab import files
import json

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: scipy 1.13.1
Uninstalling scipy-1.13.1:
  Successfully uninstalled scipy-1.13.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 149.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 172.8 MB/s eta 0:00:00


In [ ]:
uploaded = files.upload()

def load_questions(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

Saving processed_questions.json to processed_questions (1).json
Saving processed_LOs.json to processed_LOs (1).json


In [ ]:
import re
import string

def clean_text(text):
    text = re.sub(r"[\[\]]", "", text) # remove square brackets
    text = re.sub(r"(?<=\b\w)-(?=\w\b)", " ", text)  # Replace dash between words



    text = str(text).lower()  # Lowercase words
    #text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    return text


def tokenize_text(text, tokenizer, stopwords):
  tokens = tokenizer(text)  # Get tokens from text
  tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
  tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
  tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
  return tokens

In [ ]:
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import pandas as pd

questions = load_questions('processed_questions.json')
learning_outcomes = load_questions('processed_LOs.json')

data = []

for category, question_list in questions.items():
    i = 0
    for question in question_list:
        cleaned = clean_text(question)
        tokenized_question = tokenize_text(cleaned, word_tokenize, stop_words)
        data.append({
            "category": category,
            "raw_question": question,
            "tokenized_question": tokenized_question,
            "learning_outcomes": learning_outcomes[category][i]
        })
        i += 1


df = pd.DataFrame(data, columns=["category", "raw_question", "tokenized_question", "learning_outcomes"])

print(df.head(10))

                      category  \
0  Sets, Proofs, and Induction   
1  Sets, Proofs, and Induction   
2  Sets, Proofs, and Induction   
3  Sets, Proofs, and Induction   
4  Sets, Proofs, and Induction   
5  Sets, Proofs, and Induction   
6  Sets, Proofs, and Induction   
7  Sets, Proofs, and Induction   
8  Sets, Proofs, and Induction   
9  Sets, Proofs, and Induction   

                                        raw_question  \
0  Let [universal set] be the [set] of all studen...   
1  Write three descriptions of the elements of th...   
2  How many elements does each of the following s...   
3  Which of the following pairs of sets are equal...   
4  This problem concerns the following six sets:\...   
5  Let [set defined by arithmetic condition], [se...   
6  Describe in words the difference between [empt...   
7  Let [set], [set], and [set] be sets.\n(enumera...   
8  Let [range set], [finite set], and [finite set...   
9  Let [universal set], [finite set], [finite set...   

        

In [ ]:
import numpy as np

def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of tokenized documents (e.g., list of lists of strings)
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []
    # Determine if the model has 'wv' (as in custom-trained models)
    model_vectors = model.wv if hasattr(model, 'wv') else model

    for tokens in list_of_docs:
        zero_vector = np.zeros(model_vectors.vector_size)
        vectors = [model_vectors[token] for token in tokens if token in model_vectors.key_to_index]

        if vectors:
            avg_vec = np.mean(vectors, axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

In [ ]:
pip install hdbscan

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, OPTICS, SpectralClustering, AgglomerativeClustering, DBSCAN, MeanShift, AffinityPropagation, Birch
from sklearn.mixture import GaussianMixture
import hdbscan
import plotly.express as px
from tabulate import tabulate

def reduce_to_2d(df, vectors):
  X = np.vstack(vectors)
  pca = PCA(n_components=10)
  X_reduced = pca.fit_transform(X)

  tsne = TSNE(n_components=2, random_state=0)
  X_2d = tsne.fit_transform(X_reduced)

  df[['x', 'y']] = X_2d

def visualize_plot(title, df, cluster_column):
    # Scatter plot using Plotly Express
    fig = px.scatter(df, x="x", y="y", color=cluster_column,
                     color_continuous_scale='viridis',
                     title=title,
                     labels={"x": "t-SNE Component 1", "y": "t-SNE Component 2", cluster_column: "Cluster"},
                     hover_data=["category", "raw_question"])

    fig.update_traces(marker=dict(size=10))  # Adjust marker size
    fig.show()

    # Loop through each cluster and plot category distribution as a pie chart
    groups = df.groupby(cluster_column)

    for cluster, group in groups:
      # Count occurrences of each category within the cluster
      category_counts = group['category'].value_counts().reset_index()
      category_counts.columns = ['Category', 'Count']

      # Calculate percentages for each category
      total_count = category_counts['Count'].sum()
      category_counts['Percentage'] = (category_counts['Count'] / total_count) * 100

      # Round values for prettier display
      category_counts['Percentage'] = category_counts['Percentage'].round(2)

      # Print the category distribution for the current cluster in a clean table
      print(f"\nCluster {cluster} Category Distribution:")
      print(tabulate(category_counts, headers='keys', tablefmt='fancy_grid', showindex=False))


def cluster_and_visualize(vectors, num_clusters, df, eps=0.5, min_samples=2):
    """
    Perform clustering on document vectors, reduce dimensions, and plot clusters.

    Args:
        vectors: List of document vectors (e.g., output from vectorize function)
        num_clusters: Number of clusters for KMeans
        df: DataFrame containing the documents

    Returns:
        DataFrame with added 'cluster', 'x', and 'y' columns for visualization
    """

    # Step 1: Convert vectors to numpy array and perform KMeans clustering
    X = np.vstack(vectors)

    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    df["cluster_kmeans"] = kmeans.fit_predict(X)

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    df["cluster_dbscan"] = dbscan.fit_predict(X)

    clustering_optics = OPTICS(min_samples=10, xi=.01, min_cluster_size=.05).fit_predict(X)
    df["cluster_optics"] = clustering_optics

    clustering_hdbscan = hdbscan.HDBSCAN(min_cluster_size=10, gen_min_span_tree=True).fit_predict(X)
    df["cluster_hdbscan"] = clustering_hdbscan

    clustering_spectral = SpectralClustering(n_clusters=num_clusters, assign_labels='discretize', random_state=42).fit_predict(X)
    df["cluster_spectral"] = clustering_spectral

    clustering_gmm = GaussianMixture(n_components=num_clusters, random_state=42).fit_predict(X)
    df["cluster_gmm"] = clustering_gmm

    clustering_agglo = AgglomerativeClustering(n_clusters=num_clusters).fit_predict(X)
    df["cluster_agglo"] = clustering_agglo

    clustering_meanshift = MeanShift().fit_predict(X)
    df["cluster_meanshift"] = clustering_meanshift

    clustering_affinity = AffinityPropagation().fit_predict(X)
    df["cluster_affinity"] = clustering_affinity

    clustering_birch = Birch(n_clusters=num_clusters).fit_predict(X)
    df["cluster_birch"] = clustering_birch


    # Step 3: Plot the clusters
    visualize_plot("KMeans Clusters of Questions", df, "cluster_kmeans")
    visualize_plot("DBSCAN Clusters of Questions", df, "cluster_dbscan")
    visualize_plot("OPTICS Clusters of Questions", df, "cluster_optics")
    visualize_plot("HDBSCAN Clusters of Questions", df, "cluster_hdbscan")
    visualize_plot("Spectral Clusters of Questions", df, "cluster_spectral")
    visualize_plot("GMM Clusters of Questions", df, "cluster_gmm")
    visualize_plot("Agglomerative Clusters of Questions", df, "cluster_agglo")
    visualize_plot("MeanShift Clusters of Questions", df, "cluster_meanshift")
    visualize_plot("AffinityPropagation Clusters of Questions", df, "cluster_affinity")
    visualize_plot("Birch Clusters of Questions", df, "cluster_birch")


    return df

In [ ]:
!pip install gensim

In [ ]:
import gensim.downloader as api

pretrained_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
vectors = vectorize(df["tokenized_question"], pretrained_model)

In [ ]:
reduce_to_2d(df, vectors)
visualize_plot("Original categories", df, "category")


Cluster Formal Logic Category Distribution:
╒══════════════╤═════════╤══════════════╕
│ Category     │   Count │   Percentage │
╞══════════════╪═════════╪══════════════╡
│ Formal Logic │      48 │          100 │
╘══════════════╧═════════╧══════════════╛

Cluster Functions Category Distribution:
╒════════════╤═════════╤══════════════╕
│ Category   │   Count │   Percentage │
╞════════════╪═════════╪══════════════╡
│ Functions  │      61 │          100 │
╘════════════╧═════════╧══════════════╛

Cluster Relations Category Distribution:
╒════════════╤═════════╤══════════════╕
│ Category   │   Count │   Percentage │
╞════════════╪═════════╪══════════════╡
│ Relations  │      43 │          100 │
╘════════════╧═════════╧══════════════╛

Cluster Sets, Proofs, and Induction Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Sets, Proofs, and Induction 

In [ ]:
num_clusters = len(questions.keys())  # Customize as needed
df = cluster_and_visualize(vectors, num_clusters, df)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.




Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Relations                   │      32 │        88.89 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │       2 │         5.56 │
├─────────────────────────────┼─────────┼──────────────┤
│ Functions                   │       2 │         5.56 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      57 │        64.77 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      21 │        23.86 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations         


Cluster -1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      56 │        40.88 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │      30 │        21.9  │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      26 │        18.98 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │      25 │        18.25 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Sets, Proofs, and Induction │       2 │          100 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category


Cluster -1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Relations                   │      39 │        30.71 │
├─────────────────────────────┼─────────┼──────────────┤
│ Functions                   │      37 │        29.13 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │      32 │        25.2  │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      19 │        14.96 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      24 │        66.67 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and


Cluster -1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      61 │        33.52 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │      48 │        26.37 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │      43 │        23.63 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      30 │        16.48 │
╘═════════════════════════════╧═════════╧══════════════╛


...


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      17 │        41.46 │
├─────────────────────────────┼─────────┼──────────────┤
│ Functions                   │      16 │        39.02 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │       7 │        17.07 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │       1 │         2.44 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      25 │        54.35 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations         


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      44 │        61.97 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      20 │        28.17 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │       6 │         8.45 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │       1 │         1.41 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      39 │        49.37 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations         


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      57 │        32.57 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │      48 │        27.43 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │      42 │        24    │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      28 │        16    │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒════════════╤═════════╤══════════════╕
│ Category   │   Count │   Percentage │
╞════════════╪═════════╪══════════════╡
│ Functions  │       1 │          100 │
╘════════════╧═════════╧══════════════╛

Cluster 2 Category Distribution:
╒════════════╤═════════╤══════════════╕
│ Category   │   Count │   Perc


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Sets, Proofs, and Induction │       2 │          100 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Sets, Proofs, and Induction │       1 │          100 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 2 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Sets, Proofs, and Induction │       1 │          100 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 3 Category Distribution:
╒════════


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      47 │        62.67 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      22 │        29.33 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │       6 │         8    │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      41 │        83.67 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │       5 │        10.2  │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations         

In [ ]:
from gensim.models import Word2Vec

own_model = Word2Vec(
    sentences=df["tokenized_question"],  # Input tokenized documents
    vector_size=100,                     # Dimensionality of word vectors
    window=5,                            # Context window size
    min_count=1,                         # Minimum word frequency
    workers=2,                           # Number of CPU cores
    seed=42                              # Seed for reproducibility
)

In [ ]:
own_model_vectors = vectorize(df["tokenized_question"], own_model)

num_clusters = len(questions.keys())  # Customize as needed
df2 = cluster_and_visualize(own_model_vectors, num_clusters, df.copy())

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_birch.py:727: ConvergenceWarning:

Number of subclusters found (1) by BIRCH is less than (4). Decrease the threshold.




Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      22 │        35.48 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │      17 │        27.42 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │      15 │        24.19 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │       8 │        12.9  │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      22 │        44.9  │
├─────────────────────────────┼─────────┼──────────────┤
│ Functions         


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      61 │        33.52 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │      48 │        26.37 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │      43 │        23.63 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      30 │        16.48 │
╘═════════════════════════════╧═════════╧══════════════╛



Cluster -1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      31 │        35.23 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │      21 │        23.86 │
├─────────────────────────────┼─────────┼──────────────┤
│ Functions                   │      21 │        23.86 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      15 │        17.05 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      10 │        66.67 │
├─────────────────────────────┼─────────┼──────────────┤
│ Functions        


Cluster -1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      24 │        37.5  │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      15 │        23.44 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │      14 │        21.88 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │      11 │        17.19 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      13 │        46.43 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations        

...


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      22 │           44 │
├─────────────────────────────┼─────────┼──────────────┤
│ Functions                   │      14 │           28 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │       8 │           16 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │       6 │           12 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      24 │        35.29 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations         


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Relations                   │      21 │        41.18 │
├─────────────────────────────┼─────────┼──────────────┤
│ Functions                   │      12 │        23.53 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      10 │        19.61 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │       8 │        15.69 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      15 │        68.18 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and 


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      43 │        30.28 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │      42 │        29.58 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │      35 │        24.65 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      22 │        15.49 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      18 │         45   │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic      


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Relations                   │       7 │        53.85 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │       6 │        46.15 │
╘═════════════════════════════╧═════════╧══════════════╛

Cluster 1 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Formal Logic                │      10 │        58.82 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │       3 │        17.65 │
├─────────────────────────────┼─────────┼──────────────┤
│ Functions                   │       3 │        17.65 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations         


Cluster 0 Category Distribution:
╒═════════════════════════════╤═════════╤══════════════╕
│ Category                    │   Count │   Percentage │
╞═════════════════════════════╪═════════╪══════════════╡
│ Functions                   │      61 │        33.52 │
├─────────────────────────────┼─────────┼──────────────┤
│ Formal Logic                │      48 │        26.37 │
├─────────────────────────────┼─────────┼──────────────┤
│ Relations                   │      43 │        23.63 │
├─────────────────────────────┼─────────┼──────────────┤
│ Sets, Proofs, and Induction │      30 │        16.48 │
╘═════════════════════════════╧═════════╧══════════════╛


In [ ]:
def list_cluster_LOs(df, cluster_type):
  groups = df.groupby(cluster_type)
  for group, group_df in groups:
    print(f"\nCluster {group}")
    for LOs in group_df["learning_outcomes"]:
      for LO in LOs:
        print(LO)

In [ ]:
list_cluster_LOs(df, "cluster_affinity")


 Cluster 0
Given the definition of a prime number, construct a proof by contradiction for the primality of a specific number by assuming it is composite and deriving a logical inconsistency.
Given a number to test for primality, systematically examine all possible divisors and demonstrate that none satisfy the conditions for being a proper divisor, thereby confirming its prime status.
Given the definition of rational numbers, construct a proof by contradiction for the irrationality of a specific square root by assuming it is rational and deriving a contradiction about the nature of its numerator and denominator.
Given a square root to analyze, apply the method of infinite descent or other number-theoretic techniques to demonstrate that it cannot be expressed as a ratio of integers in lowest terms.

 Cluster 1
Given a statement about boolean algebra identities, construct a bidirectional proof by demonstrating logical equivalence between complex boolean expressions through systematic ap